<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0613/0613_02_2_LSTMandGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense

# 2. Keras로 LSTM 구현하기

https://keras.io/api/layers/recurrent_layers/lstm/

사실 실제로 SimpleRNN이 사용되는 경우는 거의 없습니다. 이보다는 LSTM이나 GRU을 주로 사용하는데, 이번에는 임의의 입력에 대해서 LSTM을 사용할 경우를 보겠습니다.

In [19]:
# 품사 맞추는 문제
John = [1,0,0,0]
loves = [0,1,0,0]
Jane = [0,0,1,0]
Alex = [0,0,0,1]

train_X = np.array([
    [ John, loves, Jane ],
    [ Jane, loves, Alex ]
]).astype(np.float32)

S = [0] # subject
V = [1] # verb
O = [2] # object

idx2tag = ['S', 'V', 'O']

train_Y = np.array([[S, V, O], [S, V, O]]).astype(np.float32)

print("train_y", train_Y)
print("train_X의 shape", train_X.shape)
print("train_Y의 shape", train_Y.shape)

train_y [[[0.]
  [1.]
  [2.]]

 [[0.]
  [1.]
  [2.]]]
train_X의 shape (2, 3, 4)
train_Y의 shape (2, 3, 1)


In [5]:
train_X.shape

(2, 3, 4)

In [6]:
lstm = LSTM(10)
output = lstm(train_X)

In [7]:
output # 마지막 시점의 hidden state 출력

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.00882178, -0.07148983,  0.0270995 ,  0.06419743,  0.00447996,
        -0.04266049, -0.09275564, -0.06368592,  0.08546617, -0.1283861 ],
       [-0.09605565, -0.06962553,  0.0360249 , -0.02886875,  0.04161034,
         0.06923639, -0.09273029, -0.07247909,  0.00335787, -0.15516956]],
      dtype=float32)>

In [8]:
lstm = LSTM(10, return_sequences = True)
# return_sequences = True: 전체 시점 hidden state 기억
output = lstm(train_X)

In [9]:
output # 2 : 문장개수, 3: 문장 내 단어 개수 , 10 : 벡터크기
# 모든 hidden state 기억

<tf.Tensor: shape=(2, 3, 10), dtype=float32, numpy=
array([[[-0.08567462,  0.01310028,  0.07404231,  0.07413147,
          0.03157669, -0.04797008, -0.01235444, -0.09060828,
          0.02186766,  0.09455818],
        [-0.12147957,  0.05490887, -0.00975332,  0.0514985 ,
          0.07102291, -0.12763576,  0.07175852, -0.12585807,
          0.02047815,  0.12104332],
        [-0.10094232, -0.02099633, -0.07261018, -0.01461274,
          0.09776165, -0.08455842,  0.09038998, -0.17359683,
         -0.00709027,  0.03701221]],

       [[ 0.01283624, -0.0566365 , -0.05810475, -0.03535709,
          0.04424312,  0.00575065,  0.026756  , -0.08950444,
         -0.04289158, -0.04234008],
        [-0.07094418,  0.0069267 , -0.08463157, -0.01376065,
          0.07221735, -0.08838222,  0.11074585, -0.13533238,
         -0.03022134,  0.0549912 ],
        [-0.05798961,  0.01057256, -0.13269782, -0.101689  ,
          0.01003129, -0.0546943 ,  0.09848986, -0.18504518,
         -0.02230441,  0.01970537]

In [10]:
lstm = LSTM(10, return_sequences = True, return_state = True)
# return_state = True : LSTM에 있는 hidden, cell state의 마지막 시점을 보여준다
# (참고 : simple RNN에서는 Cell state가 없어서 hidden state만 출력해줬음)
output = lstm(train_X)

In [11]:
output
# 1번째 : 전체 시점의 hidden state
# 2번째 : output의 마지막 값과 모양이 같다 = 마지막 시점의 hidden state
# 3번째 : 마지막 시점의 Cell state
# 전체 t의 ht,  # 마지막 시점 ht, # 마지막 시점의 Ct

[<tf.Tensor: shape=(2, 3, 10), dtype=float32, numpy=
 array([[[-0.10985142, -0.07429689, -0.07918032, -0.06295913,
          -0.04264182, -0.01185398, -0.06610213,  0.07219309,
          -0.02882399, -0.0522147 ],
         [-0.00578104,  0.03338828, -0.10992517,  0.00506607,
          -0.10579371,  0.04172195, -0.05331444, -0.01089991,
           0.01014715,  0.04995363],
         [-0.08640608,  0.08109606, -0.05255764,  0.04829006,
          -0.05821335,  0.06525791, -0.11046748, -0.04840568,
          -0.06078831, -0.03905934]],
 
        [[-0.07460481,  0.05860521,  0.02743198,  0.03737866,
          -0.00554104,  0.0335271 , -0.0683361 , -0.04420718,
          -0.06454726, -0.07724275],
         [ 0.00804293,  0.13052648, -0.04538567,  0.08557821,
          -0.09587607,  0.06723301, -0.05096405, -0.08732398,
          -0.02577588,  0.03248907],
         [ 0.0538917 ,  0.12758343, -0.05072109,  0.02944586,
          -0.03972085,  0.03893657, -0.10670094, -0.11267778,
          -0.04

In [14]:
output[0] # 전체 t의 ht , 2개 인건 문장이 2개라서

<tf.Tensor: shape=(2, 3, 10), dtype=float32, numpy=
array([[[-0.10985142, -0.07429689, -0.07918032, -0.06295913,
         -0.04264182, -0.01185398, -0.06610213,  0.07219309,
         -0.02882399, -0.0522147 ],
        [-0.00578104,  0.03338828, -0.10992517,  0.00506607,
         -0.10579371,  0.04172195, -0.05331444, -0.01089991,
          0.01014715,  0.04995363],
        [-0.08640608,  0.08109606, -0.05255764,  0.04829006,
         -0.05821335,  0.06525791, -0.11046748, -0.04840568,
         -0.06078831, -0.03905934]],

       [[-0.07460481,  0.05860521,  0.02743198,  0.03737866,
         -0.00554104,  0.0335271 , -0.0683361 , -0.04420718,
         -0.06454726, -0.07724275],
        [ 0.00804293,  0.13052648, -0.04538567,  0.08557821,
         -0.09587607,  0.06723301, -0.05096405, -0.08732398,
         -0.02577588,  0.03248907],
        [ 0.0538917 ,  0.12758343, -0.05072109,  0.02944586,
         -0.03972085,  0.03893657, -0.10670094, -0.11267778,
         -0.04459288, -0.00819369]

In [ ]:
output[0][:, -1]

In [ ]:
output[1] # 마지막 시점 ht

In [ ]:
output[2] # 마지막 시점의 Ct

In [ ]:
num_classes = 3
input_dim = 4
sequence_length = 3
learning_rate = 0.1


lstm = LSTM(num_classes)
output = lstm(train_X)

print('hidden state : {}, shape: {}'.format(output, output.shape))

In [ ]:
# # return_sequences = True

# lstm = LSTM(3, return_sequences=True, return_state=True)
# whole_seq_output, final_memory_state, final_carry_state = lstm(train_X)

# print('whole_seq_output: {}, shape: {}'.format(whole_seq_output, whole_seq_output.shape))
# print('final_memory_state : {}, shape: {}'.format(final_memory_state, final_memory_state.shape))
# print('final_carry_state : {}, shape: {}'.format(final_carry_state, final_carry_state.shape))
# return_sequences = True

lstm = LSTM(3, return_sequences=True, return_state=True)
whole_seq_output, final_hidden_state, final_cell_state = lstm(train_X)

print('whole_seq_output: {}, shape: {}'.format(whole_seq_output, whole_seq_output.shape))
print('final_hidden_state : {}, shape: {}'.format(final_hidden_state, final_hidden_state.shape))
print('final_cell_state : {}, shape: {}'.format(final_cell_state, final_cell_state.shape))

In [15]:
from tensorflow.keras import layers, models

model = models.Sequential() #모델 호출
model.add(
    layers.LSTM(units= 5,
                input_shape = (3,4),
                return_sequences = True,
                name='LSTM-1'))

model.add(
    layers.Dense(
        units=3,
        activation='softmax',
        name='hidden-to-output')) # 출력을 위한 FFN


model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer= 'adam',
    metrics=['accuracy'])


In [ ]:
model.fit(train_X, train_Y, epochs= 100)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.0693 - accuracy: 0.8333
Epoch 2/100
1/1 [==============================] - 0s 14ms/step - loss: 1.0635 - accuracy: 0.8333
Epoch 3/100
1/1 [==============================] - 0s 12ms/step - loss: 1.0577 - accuracy: 0.8333
Epoch 4/100
1/1 [==============================] - 0s 20ms/step - loss: 1.0517 - accuracy: 0.8333
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: 1.0455 - accuracy: 0.8333
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 1.0392 - accuracy: 0.8333
Epoch 7/100
1/1 [==============================] - 0s 9ms/step - loss: 1.0326 - accuracy: 0.8333
Epoch 8/100
1/1 [==============================] - 0s 11ms/step - loss: 1.0259 - accuracy: 0.8333
Epoch 9/100
1/1 [==============================] - 0s 11ms/step - loss: 1.0189 - accuracy: 0.8333
Epoch 10/100
1/1 [==============================] - 0s 9ms/step - loss: 1.0117 - accuracy: 0.8333
Epoch 11/100
1/1 [======

In [ ]:
predictions = model.predict(train_X)
for i, prediction in enumerate(predictions):
  print(prediction)
  print(np.argmax(prediction, axis=1))
  result_str = [idx2tag[c] for c in np.argmax(prediction, axis=1) ]
  print("\tPrediction str: ", "".join(result_str))


[[0.8940147  0.03449708 0.07148813]
 [0.04685312 0.9033953  0.04975158]
 [0.11443406 0.04352104 0.8420449 ]]
[0 1 2]
	Prediction str:  SVO
[[0.8195169  0.04282429 0.13765882]
 [0.03784835 0.90291107 0.05924062]
 [0.07242088 0.04501116 0.88256794]]
[0 1 2]
	Prediction str:  SVO


모델을 이해하려면 weight 개수(W0\~Wn)를 세어봐야 합니다.

In [16]:
model.summary()
# activation(LSTM에서는 activation이 계속 바뀌어서 함수를 따로 써주겠다)
# ht = activation(Wx * Xt + Wh*h(t-1) + b)
# 5*1 = activation(5*4 * 4*1)+ (5*5 * 5*1) + 5*1) # units= 5 줘서 5*1
# param = 5*4 + 5*5 + 5*1  = 50 * activation = 50 * 4(activation가 시그모이드 3번 tanh가 1번이라 총 4번 = 4) = 200
# RNN 보다 weight 개수가 4배다

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 LSTM-1 (LSTM)               (None, 3, 5)              200       
                                                                 
 hidden-to-output (Dense)    (None, 3, 3)              18        
                                                                 
Total params: 218
Trainable params: 218
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for model_weight in model.weights:
    print(model_weight.name, '=>', model_weight.shape)

### 문제1

In [ ]:
model = Sequential()
model.add(LSTM(7, input_shape=(100,5)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 7)                 364       
                                                                 
Total params: 364
Trainable params: 364
Non-trainable params: 0
_________________________________________________________________


- W_forget : (num_units + input_dim + 1) * num_units
- W_input : (num_units + input_dim + 1) * num_units
- W_output : (num_units + input_dim + 1) * num_units
- W_cell : (num_units + input_dim + 1) * num_units

### 문제2

In [ ]:
model = Sequential()
model.add(LSTM(5, input_shape = (2, 10)))
model.add(Dense(1))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 5)                 320       
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 326
Trainable params: 326
Non-trainable params: 0
_________________________________________________________________


- W_forget : (num_units + input_dim + 1) * num_units
- W_input : (num_units + input_dim + 1) * num_units
- W_output : (num_units + input_dim + 1) * num_units
- W_cell : (num_units + input_dim + 1) * num_units

In [ ]:
# weight 개수 카운팅

### 문제3

In [17]:
model = Sequential()
model.add(LSTM(5, input_shape = (2, 10), return_sequences=True))
model.add(LSTM(7))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 2, 5)              320       
                                                                 
 lstm_4 (LSTM)               (None, 7)                 364       
                                                                 
 dense (Dense)               (None, 1)                 8         
                                                                 
Total params: 692
Trainable params: 692
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 첫번째 레이어의 파라미터 개수
# ht = activation(Wx * Xt + Wh*h(t-1) + b)
# 5*1 = 4*(((5*10) *(10*1)) + ((5*5) * (5*1)) + (5*1)) = 50+30 = 80 *4 = 320
# output : none, 2, 5

In [ ]:
# 두번째 레이어의 파라미터 개수
# ht = activation(Wx * Xt + Wh*h(t-1) + b)
# 7*1 = 4*(((7*5) *(5*1)) + ((7*7) * (7*1)) + (7*1)) (35+49+7) *4 = 364
# output : none, 7 # return true아니라서 7까지만 출력

In [ ]:
# 세번째 레이어의 파라미터 개수
# print((activation 없음)>>  (Wx * x) + b >> ((1*7) *(7*1)) + (1*1) >> =7+1 =8
# output : none, 1

# 3. Keras로 GRU 구현하기

https://keras.io/api/layers/recurrent_layers/gru/

In [21]:
train_X

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.]]], dtype=float32)

In [ ]:
gru = GRU(10)
output = gru(train_X)

In [ ]:
output # 마지막 시점의 Hidden vector 가 출력

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.25510576,  0.14532486,  0.16247232, -0.06723822,  0.10383694,
        -0.12867996,  0.07351287,  0.03705239, -0.19907582, -0.02601814],
       [ 0.23845002,  0.3301523 ,  0.13880032,  0.08107516,  0.22277805,
        -0.18187267, -0.18029368, -0.20013377, -0.13222887,  0.03550896]],
      dtype=float32)>

In [24]:
gru = GRU(10, return_sequences=True)
output = gru(train_X)

In [25]:
output # 모든 시점의 Hidden vector 가 출력

<tf.Tensor: shape=(2, 3, 10), dtype=float32, numpy=
array([[[-0.06485951, -0.1335059 , -0.10532144,  0.09507397,
          0.07057574,  0.03021935, -0.05934305, -0.15715463,
         -0.08967832, -0.10092197],
        [-0.11201885,  0.07808554, -0.241065  ,  0.06933547,
          0.15301757,  0.13467762,  0.02141472, -0.17853157,
         -0.10067677, -0.08699638],
        [ 0.05858942,  0.17569372,  0.03951798,  0.03153273,
          0.15760893,  0.1431498 ,  0.05959178, -0.01257851,
          0.10036712, -0.08597995]],

       [[ 0.13442338,  0.13162382,  0.14367786,  0.04126311,
          0.10143887,  0.08337055,  0.0380693 ,  0.07523782,
          0.14650837, -0.02565111],
        [-0.00231821,  0.21326742, -0.1105819 ,  0.07830205,
          0.16378188,  0.1808161 ,  0.08687209, -0.1046818 ,
          0.04808076, -0.02992109],
        [ 0.07103775,  0.17564774,  0.1126399 , -0.1268832 ,
          0.02420799, -0.02393562,  0.01753471,  0.01738961,
          0.02839365, -0.08465049]

In [26]:
gru = GRU(10, return_sequences=True, return_state=True)
output = gru(train_X)

In [27]:
output # 모든 시점의 Hidden vector , 두번째는 마지막 시점의 Hidden state출력(CELL STATE 없어서 출력 안 됨)

[<tf.Tensor: shape=(2, 3, 10), dtype=float32, numpy=
 array([[[ 0.18438318,  0.08697686, -0.18993029, -0.09965076,
          -0.04078727, -0.08273635, -0.16549124,  0.13684937,
           0.11966605,  0.07066972],
         [ 0.13169135,  0.07750485, -0.21429253,  0.10722788,
           0.00731747,  0.09138015, -0.22116654, -0.03568701,
          -0.13551486, -0.08909343],
         [ 0.04103464, -0.13933074, -0.30667865, -0.07610669,
          -0.04550781,  0.07511689, -0.04365234,  0.01984416,
          -0.14235719, -0.18737191]],
 
        [[ 0.01067854, -0.16299233, -0.22308521, -0.16681366,
          -0.04624448,  0.03072454,  0.07662097,  0.03962259,
          -0.08012903, -0.18264744],
         [ 0.01651171, -0.04118147, -0.23682374,  0.05924007,
           0.00626924,  0.16529144, -0.09806661, -0.0763464 ,
          -0.19214824, -0.18765046],
         [-0.22930133, -0.04102037, -0.15229234,  0.08601587,
          -0.16825573,  0.06054401, -0.08505721, -0.10695049,
           0.01

In [20]:
gru = GRU(num_classes)
output = gru(train_X)

print('hidden state : {}, shape: {}'.format(output, output.shape))

In [ ]:
# return_sequences = True
gru = GRU(3, return_sequences=True, return_state=True)
whole_sequence_output, final_state= gru(train_X)

print('whole_seq_output: {}, shape: {}'.format(whole_seq_output, whole_seq_output.shape))
print('final_state : {}, shape: {}'.format(final_state, final_state.shape))

whole_seq_output: [[[-0.09133016 -0.02071412  0.14170164]
  [ 0.02361282  0.06159054  0.03471249]
  [ 0.06012786 -0.04471743 -0.08892121]]

 [[ 0.03587217 -0.0941782  -0.11587585]
  [ 0.08149255  0.02020295 -0.18497597]
  [ 0.03355836 -0.02213107 -0.14921774]]], shape: (2, 3, 3)
final_state : [[ 0.10870809  0.3283546  -0.2419539 ]
 [-0.13706805 -0.10124536 -0.31973392]], shape: (2, 3)


In [29]:
from tensorflow.keras import layers, models

model = models.Sequential() #모델 호출
model.add(
    layers.GRU(
        units=5,
        input_shape =(3, 4),
        return_sequences = True,
        reset_after=False,# 논문에 있는 버전으로 해달라 (논문 x = True)
        name='GRU-1')) # RNN 호출

# reset_after : keras 구현을 하면서 병렬처리를 위해 공식을 수정(bias를 2개로 나눔)하였는데, 원래 논문에 나온 공식으로 계산하기 위해 False로 수정

model.add(
    layers.Dense(
        units=3,
        activation='softmax',
        name='hidden-to-output')) # 출력을 위한 FFN


model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=['accuracy'])


In [ ]:
model.fit(train_X, train_Y, epochs= 100)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 1.1492 - accuracy: 0.1667
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 1.1400 - accuracy: 0.1667
Epoch 3/100
1/1 [==============================] - 0s 8ms/step - loss: 1.1317 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 8ms/step - loss: 1.1240 - accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 9ms/step - loss: 1.1170 - accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 8ms/step - loss: 1.1105 - accuracy: 0.1667
Epoch 7/100
1/1 [==============================] - 0s 11ms/step - loss: 1.1044 - accuracy: 0.1667
Epoch 8/100
1/1 [==============================] - 0s 13ms/step - loss: 1.0985 - accuracy: 0.3333
Epoch 9/100
1/1 [==============================] - 0s 15ms/step - loss: 1.0929 - accuracy: 0.5000
Epoch 10/100
1/1 [==============================] - 0s 10ms/step - loss: 1.0874 - accuracy: 0.6667
Epoch 11/100
1

In [ ]:
predictions = model.predict(train_X)
for i, prediction in enumerate(predictions):
  print(prediction)
  print(np.argmax(prediction, axis=1))
  result_str = [idx2tag[c] for c in np.argmax(prediction, axis=1) ]
  print("\tPrediction str: ", "".join(result_str))


[[0.8838949  0.03892433 0.07718088]
 [0.03276091 0.89175344 0.07548558]
 [0.12529185 0.05806499 0.8166431 ]]
[0 1 2]
	Prediction str:  SVO
[[0.8183433  0.04454137 0.13711534]
 [0.03734664 0.8973856  0.06526773]
 [0.07024387 0.06087027 0.8688859 ]]
[0 1 2]
	Prediction str:  SVO


모델을 이해하려면 weight 개수(W0\~Wn)를 세어봐야 합니다.

In [ ]:
# GRU는 RNN와 LSTM 사이에 있다
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 GRU-1 (GRU)                 (None, 3, 3)              72        
                                                                 
 hidden-to-output (Dense)    (None, 3, 3)              12        
                                                                 
Total params: 84
Trainable params: 84
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for model_weight in model.weights:
    print(model_weight.name, '=>', model_weight.shape)

### 문제1

In [ ]:
model = Sequential()
model.add(GRU(9, input_dim = 10, return_sequences=True, reset_after=False))
# reset_after : keras 구현을 하면서 병렬처리를 위해 공식을 수정(bias를 2개로 나눔)하였는데, 원래 논문에 나온 공식으로 계산하기 위해 False로 수정
model.add(GRU(6, reset_after = False))
model.add(Dense(3))
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_6 (GRU)                 (None, None, 9)           540       
                                                                 
 gru_7 (GRU)                 (None, 6)                 288       
                                                                 
 dense_6 (Dense)             (None, 3)                 21        
                                                                 
Total params: 849
Trainable params: 849
Non-trainable params: 0
_________________________________________________________________


- W_reset : (num_units + input_dim + 1) * num_units
- W_update : (num_units + input_dim + 1) * num_units
- W_new : (num_units + input_dim + 1) * num_units


In [ ]:
# 첫번째 레이어의 파리미터 개수

In [ ]:
# 두번째 레이어의 파라미터 개수

In [ ]:
# 세번째 레이어의 파리미터 개수

### reference

- https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/main/08.%20RNN/8-4.%20understanding_simplernn_and_lstm.ipynb